In [93]:
import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode
import torch
import pickle as pkl


In [61]:
path = "../informatie/apple_disease_classification/images/Train/Dataset/normal_apples/good_apple0.jpg"
img = read_image(path, ImageReadMode.UNCHANGED)/255

print(img.shape)
print(img)

torch.Size([3, 128, 128])
tensor([[[0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         [0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         [0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.9490, 0.9490, 0.9490,  ..., 0.9725, 0.9725, 0.9725],
         [0.9529, 0.9529, 0.9529,  ..., 0.9725, 0.9725, 0.9725],
         [0.9529, 0.9529, 0.9529,  ..., 0.9725, 0.9725, 0.9725]],

        [[0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         [0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         [0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.9412, 0.9412, 0.9412,  ..., 0.9529, 0.9529, 0.9529],
         [0.9451, 0.9451, 0.9451,  ..., 0.9529, 0.9529, 0.9529],
         [0.9451, 0.9451, 0.9451,  ..., 0.9529, 0.9529, 0.9529]],

        [[0.9569, 0.9569, 0.9569,  ..., 0.9686, 0.9686, 0.9686],
         [0.9569, 0.9569, 0.9569,  ..., 0.9686, 0.9686, 0.9686],
         [0.9569, 0.9569, 0.9569

In [62]:

dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"

In [11]:
class DatasetAppels(Dataset):
    def __init__(self, img_folder_path, transform):

        image_folder = ImageFolder(img_folder_path, transform=transform)
        len_classes = len(image_folder.class_to_idx)
        self.images = [image[0] for image in image_folder]
        self.labels = [torch.Tensor(self.labeler(image[1], len_classes)) for image in image_folder]
        self.class_dict = image_folder.class_to_idx

    def labeler(self, label, len_classes):
        result = [0] * len_classes
        result[label] = 1
        return result

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return [self.images[idx], self.labels[idx]]
    


In [63]:
dataset = DatasetAppels(dataset_path, T.ToTensor())

dataset

In [64]:
resize = T.Resize((64,64))

for i in range(len(dataset.images)):
    dataset.images[i] = resize(dataset.images[i])

print(dataset.images[0].shape)

torch.Size([3, 64, 64])


In [65]:
print(dataset.labels)
print(dataset.class_dict)

[tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1.,

In [66]:
print(dataset.images[0].type())
print(dataset.images[1000].type())
print(dataset.images[1700].type())
print(dataset.images[2200].type())

torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor


In [67]:
generator1 = torch.Generator().manual_seed(13)

# create a train test split with 70% train, 30% test
train_dataset, test_dataset, val_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=generator1)

# check length of train and test dataset
print(len(train_dataset), len(test_dataset), len(val_dataset))

# create train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

1794 598 597


In [68]:
class CNN(nn.Module):
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    def __init__(self, img_size=64, c_in=3, loss_func=nn.CrossEntropyLoss()):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(c_in, 16, 3, padding=1),
            nn.ReLU(),
            nn. Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn. Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(img_size*128, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 4),
            nn.Softmax(dim=1)
            )
        
        self.loss_func = loss_func
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
    def fit(self, train_loader, val_loader, test_loader, epochs, lr, opt_function=torch.optim.Adam):
        optimizer = opt_function(self.model.parameters(),lr )
        history = []
        self.model.to(self.cuda_available())

        for epoch in range(epochs):
            print("epoch:",epoch+1)
            self.model.train()
            
            for batch in train_loader:
                optimizer.zero_grad()
                loss = self.loss_calc(batch)
                loss.backward()
                optimizer.step()
                
            with torch.no_grad():    
                self.model.eval()

                val_loss = []
                for batch in val_loader:
                    loss = self.loss_calc(batch)
                    val_loss.append(loss)  
            
            history.append(sum(val_loss))
            print(sum(val_loss))
        
            print("Accuracy:",self.evaluate_accuracy(test_loader))
        return history
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    def cuda_available(self):

        if torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                
    def loss_calc(self, batch):
        image = batch[0].to(self.cuda_available())
        labels = batch[1].to(self.cuda_available())
        pred = self(image)        
        loss = self.loss_func(pred, labels)
        return loss
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    def evaluate_accuracy(self, test_loader):         
        cor_pred = 0
        bad_pred = 0
        for batch in test_loader:
            image, labels = batch
            image = image.to(self.cuda_available())
            labels = labels.to(self.cuda_available())
            pred = self(image)
    
            _, y_pred = torch.max(pred,1)                       
            
            for i in range(len(y_pred)):
                result = [0] * len(pred[0])

                p = int(y_pred[i])
                result[p] = 1
                if result == labels[i].tolist():
                    cor_pred += 1
                else:
                    bad_pred += 1

        acc = cor_pred/(cor_pred + bad_pred) * 100
        return acc 
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
    
    def predict(self, image):
        image = image.to(self.cuda_available())
        pred = self(image)
        print(pred)
        
        _, y_pred = torch.max(pred,1)
        print(f"y_pred: {y_pred}")                       
        
        result = [0] * len(pred[0])
        print(f"len(pred[0]): {len(pred[0])}")

        p = int(y_pred[0])
        print(f"p: {p}")
        result[p] = 1

        return result
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    # def predict(self, image):
    #     image = image.to(self.cuda_available())
    #     pred = self(image)
    #     _, y_pred = torch.max(pred,1)
    #     print(pred)
    #     print(f"y_pred: {y_pred}")                       
        
    #     for i in range(len(y_pred)):
    #         result = [0] * len(pred[0])
    #         print(f"len(pred[0]): {len(pred[0])}")

    #         p = int(y_pred[i])
    #         print(f"p: {p}")
    #         result[p] = 1

    #     return result

    

        


    
    def forward(self, data):

        return self.model(data)


In [92]:
# Loop over dataloader and push images and labels to device
# dataset.images.to("cuda")
# dataset.labels.to("cuda")

net = CNN()

net.fit(train_loader, val_loader, test_loader, lr = 0.00015, epochs=20)
net.fit(train_loader, val_loader, test_loader, lr = 0.00003, epochs=10)


epoch: 1
tensor(142.2066, device='cuda:0')
Accuracy: 60.200668896321076
epoch: 2
tensor(140.3283, device='cuda:0')
Accuracy: 60.03344481605352
epoch: 3
tensor(132.7505, device='cuda:0')
Accuracy: 66.72240802675586
epoch: 4
tensor(144.6450, device='cuda:0')
Accuracy: 52.675585284280935
epoch: 5
tensor(126.0354, device='cuda:0')
Accuracy: 70.90301003344482
epoch: 6
tensor(125.3383, device='cuda:0')
Accuracy: 71.40468227424749
epoch: 7
tensor(123.9041, device='cuda:0')
Accuracy: 72.5752508361204
epoch: 8
tensor(126.8973, device='cuda:0')
Accuracy: 69.39799331103679
epoch: 9
tensor(121.9486, device='cuda:0')
Accuracy: 74.41471571906354
epoch: 10
tensor(120.5148, device='cuda:0')
Accuracy: 75.4180602006689
epoch: 11
tensor(119.9213, device='cuda:0')
Accuracy: 74.74916387959865
epoch: 12
tensor(119.2619, device='cuda:0')
Accuracy: 76.58862876254182
epoch: 13
tensor(119.2550, device='cuda:0')
Accuracy: 76.58862876254182
epoch: 14
tensor(120.5484, device='cuda:0')
Accuracy: 75.25083612040135
e

[tensor(113.5486, device='cuda:0'),
 tensor(113.5649, device='cuda:0'),
 tensor(113.1128, device='cuda:0'),
 tensor(113.0336, device='cuda:0'),
 tensor(112.8461, device='cuda:0'),
 tensor(112.8431, device='cuda:0'),
 tensor(112.7750, device='cuda:0'),
 tensor(112.5669, device='cuda:0'),
 tensor(112.6349, device='cuda:0'),
 tensor(113.0372, device='cuda:0')]

In [95]:
model_path = "..//models/CNN_83"

with open(model_path, 'wb') as f:
        pkl.dump(net, f)

In [99]:
load = open(model_path, "rb")
loaded_net = pkl.load(load)

normal = [120,121,122,123]

for i in normal:
    test_path = f"../informatie/apple_disease_classification/images/Test/blotch_Apple/{i}.jpg"
    test_img = read_image(test_path, ImageReadMode.UNCHANGED)/255

    test_img = resize(test_img)
    print(net(test_img.unsqueeze(0).to("cuda")))
    test_img = test_img.unsqueeze(0)
    result = loaded_net.predict(test_img)
    print(result)
    # test_img.shape
    

tensor([[9.9409e-01, 6.0565e-10, 4.2505e-07, 5.9122e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[9.9409e-01, 6.0565e-10, 4.2505e-07, 5.9122e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
y_pred: tensor([0], device='cuda:0')
len(pred[0]): 4
p: 0
[1, 0, 0, 0]
tensor([[8.2813e-01, 4.7337e-08, 1.7187e-01, 1.4833e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[8.2813e-01, 4.7337e-08, 1.7187e-01, 1.4833e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
y_pred: tensor([0], device='cuda:0')
len(pred[0]): 4
p: 0
[1, 0, 0, 0]
tensor([[9.9821e-01, 3.4643e-08, 5.3012e-05, 1.7360e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[9.9821e-01, 3.4643e-08, 5.3012e-05, 1.7360e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
y_pred: tensor([0], device='cuda:0')
len(pred[0]): 4
p: 0
[1, 0, 0, 0]
tensor([[5.4494e-01, 2.1900e-10, 9.5030e-07, 4.5506e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[5

In [57]:
import os
os.getcwd()

'c:\\Users\\nilsm\\workspace\\MakeAIWork3\\Project 3\\notebooks'

In [ ]:
net.predict(test_img)

tensor([[8.3434e-04, 4.0685e-09, 9.9917e-01, 4.1260e-08]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
y_pred: tensor([2], device='cuda:0')
len(pred[0]): 4
p: 2


[0, 0, 1, 0]

In [ ]:
# def evaluate_accuracy(self, test_loader):         
#         cor_pred = 0
#         bad_pred = 0
#         for batch in test_loader:
#             image, labels = batch
#             image = image.to(self.cuda_available())
#             labels = labels.to(self.cuda_available())
#             result = self.predict(image)

#             for i in range(len(result)):

#                 if result == labels[i].tolist():
#                     cor_pred += 1
#                 else:
#                     bad_pred += 1

#         acc = cor_pred/(cor_pred + bad_pred) * 100
#         return acc     
    
#     def predict(self, image):
#         pred = self(image)
#         _, y_pred = torch.max(pred,1)                       
        
#         for i in range(len(y_pred)):
#             result = [0] * len(pred[0])

#             p = int(y_pred[i])
#             result[p] = 1

#         return result

<h1>Trying a pretrained model (Resnet152V2)</h1>

In [102]:
# Resnet variants:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

Using cache found in C:\Users\nilsm/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\nilsm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nilsm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\nilsm/.cache\to

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

tensor([[ 3.9598e+00,  3.5746e+00,  4.3911e+00,  1.0403e+01,  7.6322e+00,
          1.2510e+01,  1.3622e+01, -9.8779e+00, -1.0966e+01, -1.1457e+01,
         -1.1623e+00, -6.4075e+00, -6.8714e+00,  2.2168e+00, -3.3824e+00,
          3.4502e-01, -6.0733e+00, -3.8418e+00,  2.3481e-01, -7.1640e+00,
         -5.6220e-01, -2.6839e+00, -2.9500e+00, -4.4038e+00, -1.8613e+00,
          9.1596e-01,  9.4481e+00,  5.2414e+00,  1.0556e+01,  2.0712e+01,
          9.8912e+00,  7.3717e+00,  3.2001e+00,  5.4528e-01, -6.3347e+00,
          9.0478e-01,  3.9063e+00, -4.9102e+00,  1.8355e+00, -2.1759e+00,
          4.4712e+00, -3.2462e+00, -9.9700e+00, -7.9604e-01, -4.5329e+00,
         -3.7040e+00,  5.1636e+00,  5.9382e+00, -3.1308e+00, -6.8928e+00,
         -3.3628e-01,  9.0688e+00, -1.5167e+00,  1.6840e+00, -3.9321e-01,
          1.1465e+01, -2.5647e+00, -3.0674e+00,  4.7294e+00,  5.5021e+00,
         -8.5412e-01,  3.8644e+00, -1.0555e+00, -5.0615e+00,  1.2482e+01,
          2.1894e+00,  4.8014e+00, -4.